# DE⫶TR -- Extending Object Detection to Panoptic Segmentation
> An Extension to Object Detection using Transformers to Predict Pixel wise mask of each class.

- toc: true 
- badges: true
- comments: true
- categories: [Custom Dataset, Object Detection, Panoptic Segmentation, COCO ,DETR]
- image: images/PanopticSegmentation/DETR.gif 

![SuperCool](../images/PanopticSegmentation/DETR.gif )